In [ ]:
import os
import subprocess
import pandas as pd
from tqdm import tqdm

def convert_pcap_to_csv(pcap_file, output_dir, cicflowmeter_path):
    """
    Convert a PCAP file to CSV using CICFlowMeter.
    
    Args:
    pcap_file (str): Path to the input PCAP file.
    output_dir (str): Directory to save the output CSV file.
    cicflowmeter_path (str): Path to the CICFlowMeter JAR file.
    
    Returns:
    str: Path to the output CSV file.
    """
    # Create output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)
    
    # Generate output file name
    output_file = os.path.join(output_dir, os.path.splitext(os.path.basename(pcap_file))[0] + '.csv')
    
    # Construct the command to run CICFlowMeter
    command = f"java -jar {cicflowmeter_path} {pcap_file} {output_dir}"
    
    # Run CICFlowMeter
    try:
        subprocess.run(command, shell=True, check=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
        print(f"Successfully converted {pcap_file} to CSV.")
        return output_file
    except subprocess.CalledProcessError as e:
        print(f"Error converting {pcap_file}: {e}")
        return None

def process_pcap_directory(pcap_dir, output_dir, cicflowmeter_path):
    """
    Process all PCAP files in a directory and convert them to CSV.
    
    Args:
    pcap_dir (str): Directory containing PCAP files.
    output_dir (str): Directory to save the output CSV files.
    cicflowmeter_path (str): Path to the CICFlowMeter JAR file.
    
    Returns:
    list: Paths to all generated CSV files.
    """
    csv_files = []
    pcap_files = [f for f in os.listdir(pcap_dir) if f.endswith('.pcap')]
    
    for pcap_file in tqdm(pcap_files, desc="Converting PCAP files"):
        full_path = os.path.join(pcap_dir, pcap_file)
        csv_file = convert_pcap_to_csv(full_path, output_dir, cicflowmeter_path)
        if csv_file:
            csv_files.append(csv_file)
    
    return csv_files

def combine_csv_files(csv_files, output_file):
    """
    Combine multiple CSV files into a single file.
    
    Args:
    csv_files (list): List of paths to CSV files.
    output_file (str): Path to the output combined CSV file.
    """
    combined_df = pd.concat([pd.read_csv(f) for f in csv_files])
    combined_df.to_csv(output_file, index=False)
    print(f"Combined CSV saved to {output_file}")

# Example usage
pcap_directory = '/path/to/pcap/files'
output_directory = '/path/to/output/csv/files'
cicflowmeter_path = '/path/to/CICFlowMeter.jar'
combined_output = '/path/to/combined_output.csv'

csv_files = process_pcap_directory(pcap_directory, output_directory, cicflowmeter_path)
combine_csv_files(csv_files, combined_output)